In [103]:
#Library

import numpy as np
import pandas as pd
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/noordeen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [104]:
#data loading
train = pd.read_excel('Data_Train.xlsx') 
test = pd.read_excel('Data_Test.xlsx') 
submission = pd.read_excel('Sample_Submission.xlsx')

In [105]:
train.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


In [106]:
test.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
0,The Complete Sherlock Holmes: 2 Boxes sets,Sir Arthur Conan Doyle,"Mass Market Paperback,– 1 Oct 1986",4.4 out of 5 stars,960 customer reviews,A collection of entire body of work of the She...,Short Stories (Books),"Crime, Thriller & Mystery"
1,Learn Docker - Fundamentals of Docker 18.x: Ev...,Gabriel N. Schenker,"Paperback,– Import, 26 Apr 2018",5.0 out of 5 stars,1 customer review,Enhance your software deployment workflow usin...,Operating Systems Textbooks,"Computing, Internet & Digital Media"
2,Big Girl,Danielle Steel,"Paperback,– 17 Mar 2011",5.0 out of 5 stars,4 customer reviews,"'Watch out, world. Here I come!'\nFor Victoria...",Romance (Books),Romance
3,Think Python: How to Think Like a Computer Sci...,Allen B. Downey,"Paperback,– 2016",4.1 out of 5 stars,11 customer reviews,"If you want to learn how to program, working w...",Programming & Software Development (Books),"Computing, Internet & Digital Media"
4,Oxford Word Skills: Advanced - Idioms & Phrasa...,Redman Gairns,"Paperback,– 26 Dec 2011",4.4 out of 5 stars,9 customer reviews,"Learn and practise the verbs, prepositions and...",Linguistics (Books),"Language, Linguistics & Writing"


In [107]:
submission.head()

,Price
0,119
1,123
2,108
3,118
4,132


In [108]:
train.shape, test.shape

((6237, 9), (1560, 8))

In [109]:
train.nunique()

Title           5568
Author          3679
Edition         3370
Reviews           36
Ratings          342
Synopsis        5549
Genre            345
BookCategory      11
Price           1614
dtype: int64

In [110]:
test.nunique()

Title           1521
Author          1224
Edition         1259
Reviews           30
Ratings          163
Synopsis        1519
Genre            225
BookCategory      11
dtype: int64

In [111]:
train.duplicated().sum()

0

In [112]:
test.duplicated().sum()

8

In [113]:
test[test.duplicated()]

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
1090,The Rustlers of the West Fork (Hopalong Cassidy),Louis L'Amour,"Mass Market Paperback,– 1 Apr 1992",5.0 out of 5 stars,2 customer reviews,"In this first of four classic frontier novels,...",Action & Adventure (Books),Action & Adventure
1095,The Princess Bride: S. Morgenstern's Classic T...,William Goldman,"Mass Market Paperback,– 26 Mar 2010",3.8 out of 5 stars,6 customer reviews,William Goldman's modern fantasy classic is a ...,Action & Adventure (Books),Action & Adventure
1103,Everybody Lies: The New York Times Bestseller,Seth Stephens-Davidowitz,"Paperback,– 18 Apr 2018",4.3 out of 5 stars,27 customer reviews,THE NEW YORK TIMES BESTSELLER\nAN ECONOMIST BO...,Political Structure & Processes (Books),Action & Adventure
1253,Flint,Louis L'Amour,"Mass Market Paperback,– 3 Nov 1997",4.3 out of 5 stars,4 customer reviews,"He left the West at the age of seventeen, leav...",Action & Adventure (Books),Action & Adventure
1316,The Five Greatest Warriors (Jack West Series),Matthew Reilly,"Paperback,– 25 Nov 2010",4.4 out of 5 stars,12 customer reviews,The end is approaching ... Can Jack West unrav...,Action & Adventure (Books),Action & Adventure
1340,Alita: Battle Angel – The Official Movie Novel...,Pat Cadigan,"Paperback,– 14 Feb 2019",3.9 out of 5 stars,7 customer reviews,The ofﬁcial novelization to the highly anticip...,Action & Adventure (Books),Action & Adventure
1407,The Curious Incident of the Dog in the Night-t...,Mark Haddon,"Paperback,– 2 Aug 2012",4.0 out of 5 stars,96 customer reviews,A Children’s Bookshelf Selection: Each month o...,Action & Adventure (Books),Action & Adventure
1453,Storm and Silence: Volume 1 (Storm and Silence...,Robert Thier,"Paperback,– 19 Mar 2016",5.0 out of 5 stars,59 customer reviews,Freedom - that is what Lilly Linton wants most...,Action & Adventure (Books),Action & Adventure


In [114]:
test[test['Title']=='Flint']

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
1171,Flint,Louis L'Amour,"Mass Market Paperback,– 3 Nov 1997",4.3 out of 5 stars,4 customer reviews,"He left the West at the age of seventeen, leav...",Action & Adventure (Books),Action & Adventure
1253,Flint,Louis L'Amour,"Mass Market Paperback,– 3 Nov 1997",4.3 out of 5 stars,4 customer reviews,"He left the West at the age of seventeen, leav...",Action & Adventure (Books),Action & Adventure


In [115]:
train['source'] = 'train'
test['source'] = 'test'

In [116]:
data = pd.concat([train,test])

In [117]:
data.isna().sum()

Title              0
Author             0
Edition            0
Reviews            0
Ratings            0
Synopsis           0
Genre              0
BookCategory       0
Price           1560
source             0
dtype: int64

In [118]:
data['Title']

0                     The Prisoner's Gold (The Hunters 3)
1                      Guru Dutt: A Tragedy in Three Acts
2                            Leviathan (Penguin Classics)
3                      A Pocket Full of Rye (Miss Marple)
4              LIFE 70 Years of Extraordinary Photography
                              ...                        
1555    100 Things Every Designer Needs to Know About ...
1556    Modern Letter Writing Course: Personal, Busine...
1557                        The Kite Runner Graphic Novel
1558      Panzer Leader (Penguin World War II Collection)
1559                        Complete Spanish Step-by-Step
Name: Title, Length: 7797, dtype: object

In [119]:
data = data.reset_index().drop(columns='index')

In [120]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train


In [121]:
#Data Cleaning

In [122]:
data['Reviews'] = data['Reviews'].apply(lambda x: float(x.split(' ')[0]))

In [123]:
data['Reviews'].mean()

4.295844555598315

In [124]:
data['Reviews'].isna().sum()

0

In [125]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train


In [126]:
data['Ratings'] = data['Ratings'].apply(lambda x: x.split(' ')[0])

In [127]:
data['Ratings'].astype(np.int)

/var/folders/02/n950_yc963qdyzpwj00gmdm40000gn/T/ipykernel_50564/1578310480.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['Ratings'].astype(np.int)


ValueError: invalid literal for int() with base 10: '1,416'

In [128]:
data['Ratings'] = data['Ratings'].apply(lambda x : x.replace(",",""))

In [129]:
data['Ratings'] = data['Ratings'].astype(np.int)

/var/folders/02/n950_yc963qdyzpwj00gmdm40000gn/T/ipykernel_50564/4201846288.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['Ratings'] = data['Ratings'].astype(np.int)


In [130]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train


In [131]:
data['Ratings'].isna().sum()

0

In [132]:
data['Publication'] = data['Edition'].apply(lambda x : x.split(",")[0])

In [133]:
data['Publication_Date'] = data['Edition'].apply(lambda x : x.split(",")[1])

In [134]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_Date
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train,Paperback,– 10 Mar 2016
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train,Paperback,– 7 Nov 2012
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train,Paperback,– 25 Feb 1982
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train,Paperback,– 5 Oct 2017
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train,Hardcover,– 10 Oct 2006


In [135]:
data['Publication'].unique()

array(['Paperback', 'Hardcover', 'Mass Market Paperback', 'Sheet music',
       'Flexibound', 'Plastic Comb', 'Loose Leaf', 'Tankobon Softcover',
       'Perfect Paperback', 'Board book', 'Cards', 'Spiral-bound',
       '(Kannada)', 'Product Bundle', 'Library Binding', '(German)',
       'Leather Bound', '(French)', '(Spanish)', '(Chinese)'],
      dtype=object)

In [136]:
data['Publication'].value_counts(dropna=False)

Paperback                6458
Hardcover                1056
Mass Market Paperback     200
Sheet music                32
Flexibound                 19
Cards                       9
Spiral-bound                6
Loose Leaf                  3
Tankobon Softcover          3
Perfect Paperback           1
Board book                  1
Plastic Comb                1
(Kannada)                   1
Product Bundle              1
Library Binding             1
(German)                    1
Leather Bound               1
(French)                    1
(Spanish)                   1
(Chinese)                   1
Name: Publication, dtype: int64

In [137]:
data['Publication'].isna().sum()

0

In [138]:
major_publication = ['Paperback','Hardcover']

In [139]:
data['Publication'] = data['Publication'].apply(lambda x: x if x in major_publication else 'other' )

In [140]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_Date
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train,Paperback,– 10 Mar 2016
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train,Paperback,– 7 Nov 2012
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train,Paperback,– 25 Feb 1982
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train,Paperback,– 5 Oct 2017
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train,Hardcover,– 10 Oct 2006


In [141]:
data['Publication_Year'] = data['Edition'].str[-4:]

In [142]:
data['Publication_Year'].unique()

array(['2016', '2012', '1982', '2017', '2006', '2009', '2018', '2015',
       '2013', '1999', '2002', '2011', '1991', '2014', '1989', '2000',
       '2005', '2019', '2008', '2004', '2010', '2007', '2001', '1969',
       '1993', '1992', '2003', '1996', 'port', '1997', '1995', 'NTSC',
       '1987', '1986', '1990', '1988', '1981', '1976', '1994', '1998',
       '1977', '1974', '1983', '1971', '1985', '1978', 'mile', ' set',
       'tion', '1964', '1984', '1980', 'dged', '1979', 'rint', '1960',
       '1970', '1975', '1905', '1900', 'book', '1961', '1925', '1973',
       'ated', '1942', '1972'], dtype=object)

In [143]:
data[data['Publication_Year']=='port']

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_Date,Publication_Year
169,Long Walk to Freedom: Illustrated Children's e...,Nelson Mandela,"Paperback,– Abridged, Import",5.0,2,Long Walk to Freedom by Nelson Mandela is the ...,Action & Adventure (Books),Action & Adventure,418.0,train,Paperback,– Abridged,port
582,Fundamentals of Drawing Portraits: A Practical...,Barrington Barber,"Paperback,– Illustrated, Import",4.0,7,This book is essential for everyone who wants ...,"Handicrafts, Decorative Arts & Crafts (Books)","Arts, Film & Photography",415.0,train,Paperback,– Illustrated,port
972,"Cartooning, The Professional Step-by-Step Guid...","Ivan Hissey, Curtis Tappenden","Paperback,– Illustrated, Import",3.0,1,A comprehensive and practical guide to drawing...,"Handicrafts, Decorative Arts & Crafts (Books)",Comics & Mangas,445.0,train,Paperback,– Illustrated,port
1233,Amma Tell Me About Raksha Bandhan!,Bhakti Mathur,"Paperback,– Large Print, Import",5.0,1,A promise to love and to protect\nIs sealed by...,Action & Adventure (Books),Action & Adventure,277.0,train,Paperback,– Large Print,port
1631,Swimming: Swimming Made Easy: Beginner and Exp...,Ace McCloud,"Paperback,– Large Print, Import",4.0,1,Are you tired of being a weak swimmer? Do less...,"Swimming, Snorkelling & Diving (Books)",Sports,709.0,train,Paperback,– Large Print,port
2779,Madhubani Art: Indian Art Series,Bharti Dayal,"Paperback,– Illustrated, Import",5.0,4,Madhubani art's origin is believed to go back ...,Painting Books,"Arts, Film & Photography",805.0,train,Paperback,– Illustrated,port
3875,While the Light Lasts (The Agatha Christie Col...,Agatha Christie,"Hardcover,– Facsimile, Import",3.5,7,"The very last Agatha Christie book, including ...",Short Stories (Books),"Crime, Thriller & Mystery",896.0,train,Hardcover,– Facsimile,port
4036,Mountaineering: The Freedom of the Hills,The Mountaineers,"Paperback,– Illustrated, Import",5.0,1,"For nearly 60 years it's been revered as the ""...",Outdoor Survival Skills (Books),Sports,1826.0,train,Paperback,– Illustrated,port
4403,Indian Tibet Tibetan India: The Cultural Legac...,Peter Van Ham,"Hardcover,– Illustrated, Import",4.4,2,Inspired by the first cultural expedition into...,Travel & Holiday Guides (Books),"Arts, Film & Photography",1542.0,train,Hardcover,– Illustrated,port
6816,"The Spy Chronicles: RAW, ISI and the Illusion ...",A.S. Dulat,"Hardcover,– Illustrated, Import",3.5,132,Pointing to the horizon where the sea and sky ...,Politics (Books),Politics,NaN,test,Hardcover,– Illustrated,port


In [144]:
data['Publication_Year'] =  data['Publication_Year'].apply(lambda x : re.sub("[^0-9]+", 'NA', x))

In [145]:
data['Publication_Year'] = data['Publication_Year'].replace('NA',0)

In [146]:
data['Publication_Year'].unique()

array(['2016', '2012', '1982', '2017', '2006', '2009', '2018', '2015',
       '2013', '1999', '2002', '2011', '1991', '2014', '1989', '2000',
       '2005', '2019', '2008', '2004', '2010', '2007', '2001', '1969',
       '1993', '1992', '2003', '1996', 0, '1997', '1995', '1987', '1986',
       '1990', '1988', '1981', '1976', '1994', '1998', '1977', '1974',
       '1983', '1971', '1985', '1978', '1964', '1984', '1980', '1979',
       '1960', '1970', '1975', '1905', '1900', '1961', '1925', '1973',
       '1942', '1972'], dtype=object)

In [147]:
data['Age_of_book'] = 2022 - data['Publication_Year'].astype(int)

In [148]:
data['Age_of_book'].unique()

array([   6,   10,   40,    5,   16,   13,    4,    7,    9,   23,   20,
         11,   31,    8,   33,   22,   17,    3,   14,   18,   12,   15,
         21,   53,   29,   30,   19,   26, 2022,   25,   27,   35,   36,
         32,   34,   41,   46,   28,   24,   45,   48,   39,   51,   37,
         44,   58,   38,   42,   43,   62,   52,   47,  117,  122,   61,
         97,   49,   80,   50])

In [149]:
data['Age_of_book'] = data['Age_of_book'].replace(2022,0)

In [150]:
data['Age_of_book'].unique()

array([  6,  10,  40,   5,  16,  13,   4,   7,   9,  23,  20,  11,  31,
         8,  33,  22,  17,   3,  14,  18,  12,  15,  21,  53,  29,  30,
        19,  26,   0,  25,  27,  35,  36,  32,  34,  41,  46,  28,  24,
        45,  48,  39,  51,  37,  44,  58,  38,  42,  43,  62,  52,  47,
       117, 122,  61,  97,  49,  80,  50])

In [151]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_Date,Publication_Year,Age_of_book
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train,Paperback,– 10 Mar 2016,2016,6
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train,Paperback,– 7 Nov 2012,2012,10
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train,Paperback,– 25 Feb 1982,1982,40
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train,Paperback,– 5 Oct 2017,2017,5
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train,Hardcover,– 10 Oct 2006,2006,16


In [152]:
data['Age_of_book'].mean()

10.067718353212774

In [153]:
data['Age_of_book'] = data['Age_of_book'].replace(0,10)

In [154]:
data['BookCategory'].nunique()

11

In [155]:
data['Title']

0                     The Prisoner's Gold (The Hunters 3)
1                      Guru Dutt: A Tragedy in Three Acts
2                            Leviathan (Penguin Classics)
3                      A Pocket Full of Rye (Miss Marple)
4              LIFE 70 Years of Extraordinary Photography
                              ...                        
7792    100 Things Every Designer Needs to Know About ...
7793    Modern Letter Writing Course: Personal, Busine...
7794                        The Kite Runner Graphic Novel
7795      Panzer Leader (Penguin World War II Collection)
7796                        Complete Spanish Step-by-Step
Name: Title, Length: 7797, dtype: object

In [156]:
stop_words = set(stopwords.words('english'))

In [157]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_Date,Publication_Year,Age_of_book
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train,Paperback,– 10 Mar 2016,2016,6
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train,Paperback,– 7 Nov 2012,2012,10
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train,Paperback,– 25 Feb 1982,1982,40
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train,Paperback,– 5 Oct 2017,2017,5
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train,Hardcover,– 10 Oct 2006,2006,16


In [158]:
def pre_process_nlp(x):
    string = ""
    x = re.sub("[^A-Za-z]", " ",x)
    for word in x.lower().split():
        if not word in stop_words:
            string += word + " "
    return string

In [159]:
pre_process_nlp("ChiRunning: A Revolutionary Approach to Effortless, Injury-Free Running")

'chirunning revolutionary approach effortless injury free running '

In [160]:
process_column = ['Title', 'Author', "Edition","Synopsis", "Genre", "BookCategory"]

for i in process_column:
    data[i] = data[i].apply(pre_process_nlp)

In [161]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_Date,Publication_Year,Age_of_book
0,prisoner gold hunters,chris kuzneski,paperback mar,4.0,8,hunters return third brilliant novel sunday ti...,action adventure books,action adventure,220.00,train,Paperback,– 10 Mar 2016,2016,6
1,guru dutt tragedy three acts,arun khopkar,paperback nov,3.9,14,layered portrait troubled genius art merely th...,cinema broadcast books,biographies diaries true accounts,202.93,train,Paperback,– 7 Nov 2012,2012,10
2,leviathan penguin classics,thomas hobbes,paperback feb,4.8,6,time men live without common power keep awe co...,international relations,humour,299.00,train,Paperback,– 25 Feb 1982,1982,40
3,pocket full rye miss marple,agatha christie,paperback oct,4.1,13,handful grain found pocket murdered businessma...,contemporary fiction books,crime thriller mystery,180.00,train,Paperback,– 5 Oct 2017,2017,5
4,life years extraordinary photography,editors life,hardcover oct,5.0,1,seven decades life thrilling world unrivalled ...,photography textbooks,arts film photography,965.62,train,Hardcover,– 10 Oct 2006,2006,16


In [162]:
data['Genre'].value_counts()

action adventure books                1183
romance books                          517
biographies autobiographies books      463
crime thriller mystery books           335
contemporary fiction books             325
                                      ... 
bodybuilding weightlifting books         1
immunology                               1
target shooting books                    1
ecotourism                               1
military history books                   1
Name: Genre, Length: 363, dtype: int64

In [163]:
data['Genre'].map(data['Genre'].value_counts())

0       1183
1         92
2         25
3        325
4         47
        ... 
7792     120
7793      19
7794     119
7795       5
7796      98
Name: Genre, Length: 7797, dtype: int64

In [164]:
process_column = ['Title', 'Author', "Edition","Synopsis", "Genre", "BookCategory"]

for i in process_column:
    data[i] = data[i].map(data[i].value_counts())


In [165]:
data

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_Date,Publication_Year,Age_of_book
0,1,4,431,4.0,8,1,1183,1036,220.00,train,Paperback,– 10 Mar 2016,2016,6
1,2,2,446,3.9,14,2,92,732,202.93,train,Paperback,– 7 Nov 2012,2012,10
2,3,3,366,4.8,6,3,25,670,299.00,train,Paperback,– 25 Feb 1982,1982,40
3,1,87,559,4.1,13,1,325,878,180.00,train,Paperback,– 5 Oct 2017,2017,5
4,1,1,117,5.0,1,1,47,638,965.62,train,Hardcover,– 10 Oct 2006,2006,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7792,2,2,442,5.0,4,2,120,648,NaN,test,Paperback,– 14 Apr 2011,2011,11
7793,2,2,486,3.6,13,2,19,732,NaN,test,Paperback,– 8 May 2013,2013,9
7794,1,2,493,4.0,5,1,119,670,NaN,test,Paperback,– 6 Sep 2011,2011,11
7795,1,1,493,3.5,3,1,5,732,NaN,test,Paperback,– 22 Sep 2009,2009,13


In [166]:
data.columns

Index(['Title', 'Author', 'Edition', 'Reviews', 'Ratings', 'Synopsis', 'Genre',
       'BookCategory', 'Price', 'source', 'Publication', 'Publication_Date',
       'Publication_Year', 'Age_of_book'],
      dtype='object')

In [167]:
data['BookCategory'].nunique()

11

In [168]:
data.dtypes

Title                 int64
Author                int64
Edition               int64
Reviews             float64
Ratings               int64
Synopsis              int64
Genre                 int64
BookCategory          int64
Price               float64
source               object
Publication          object
Publication_Date     object
Publication_Year     object
Age_of_book           int64
dtype: object

In [169]:
dat_filter = data[['Title', 'Author', 'Edition', 'Reviews', 'Ratings', 'Synopsis', 'Genre',
       'BookCategory', 'Price', 'source', 'Publication',
       'Age_of_book']]

In [170]:
dat_filter

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Age_of_book
0,1,4,431,4.0,8,1,1183,1036,220.00,train,Paperback,6
1,2,2,446,3.9,14,2,92,732,202.93,train,Paperback,10
2,3,3,366,4.8,6,3,25,670,299.00,train,Paperback,40
3,1,87,559,4.1,13,1,325,878,180.00,train,Paperback,5
4,1,1,117,5.0,1,1,47,638,965.62,train,Hardcover,16
...,...,...,...,...,...,...,...,...,...,...,...,...
7792,2,2,442,5.0,4,2,120,648,NaN,test,Paperback,11
7793,2,2,486,3.6,13,2,19,732,NaN,test,Paperback,9
7794,1,2,493,4.0,5,1,119,670,NaN,test,Paperback,11
7795,1,1,493,3.5,3,1,5,732,NaN,test,Paperback,13


In [171]:
data_filter_category = pd.get_dummies(data[['Publication']])

In [172]:
data_filter_category

,Publication_Hardcover,Publication_Paperback,Publication_other
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,1,0,0
...,...,...,...
7792,0,1,0
7793,0,1,0
7794,0,1,0
7795,0,1,0


In [173]:
data_preprocessed = pd.concat([data_filter_category,dat_filter.drop(columns = ['Publication'])], axis = 1)

In [174]:
train_preprocessed = data_preprocessed[data_preprocessed['source']=='train']

In [175]:
test_preprocessed = data_preprocessed[data_preprocessed['source']=='test']

In [176]:
train_preprocessed.drop(columns= ['source'], inplace = True)

/var/folders/02/n950_yc963qdyzpwj00gmdm40000gn/T/ipykernel_50564/2522153181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_preprocessed.drop(columns= ['source'], inplace = True)


In [177]:
test_preprocessed.drop(columns= ['source','Price'], inplace = True)

/var/folders/02/n950_yc963qdyzpwj00gmdm40000gn/T/ipykernel_50564/1573317149.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_preprocessed.drop(columns= ['source','Price'], inplace = True)


In [178]:
test_preprocessed = test_preprocessed.reset_index().drop(columns='index')

In [179]:
train_preprocessed.shape

(6237, 13)

In [180]:
test_preprocessed.shape

(1560, 12)

In [181]:
train_preprocessed.to_csv("train_preprocessed_2.csv", index = False)
test_preprocessed.to_csv("test_preprocessed_2.csv", index = False)

In [182]:
train_preprocessed.apply(lambda x : x.unique())

Publication_Hardcover                                               [0, 1]
Publication_Paperback                                               [1, 0]
Publication_other                                                   [0, 1]
Title                             [1, 2, 3, 17, 5, 4, 11, 14, 8, 7, 10, 9]
Author                   [4, 2, 3, 87, 1, 7, 13, 26, 33, 5, 15, 11, 6, ...
Edition                  [431, 446, 366, 559, 117, 486, 13, 334, 478, 5...
Reviews                  [4.0, 3.9, 4.8, 4.1, 5.0, 4.5, 4.4, 4.7, 4.2, ...
Ratings                  [8, 14, 6, 13, 1, 72, 16, 111, 132, 17, 4, 3, ...
Synopsis                                          [1, 2, 3, 5, 4, 8, 6, 7]
Genre                    [1183, 92, 25, 325, 47, 83, 335, 80, 463, 61, ...
BookCategory             [1036, 732, 670, 878, 638, 614, 733, 648, 702,...
Price                    [220.0, 202.93, 299.0, 180.0, 965.62, 900.0, 2...
Age_of_book              [6, 10, 40, 5, 16, 13, 4, 7, 9, 23, 20, 11, 31...
dtype: object

In [183]:
train_preprocessed.dtypes

Publication_Hardcover      uint8
Publication_Paperback      uint8
Publication_other          uint8
Title                      int64
Author                     int64
Edition                    int64
Reviews                  float64
Ratings                    int64
Synopsis                   int64
Genre                      int64
BookCategory               int64
Price                    float64
Age_of_book                int64
dtype: object